In [9]:
#ARCS Calculation
import numpy as np

# Data arrays
attention = np.array([4.25,4.5,4,3.5,5,4.75,3.75,3.5,4.25,4])
relevance = np.array([3.5, 4.75, 4, 4, 4.25, 4.75, 4.75, 5, 4, 3])
confidence = np.array([3.75, 3.5, 3.25, 3, 3.5, 3.25, 3.25, 3.5, 3.25, 2.75])
satisfaction = np.array([4.25, 5, 4, 3.75, 5, 4.75, 4, 4.5, 4, 4.5])
perceived_learning = np.array([4, 5, 3, 3.5, 4.75, 4.5, 3, 3.5, 4.75, 3.5])

# Function to calculate mean, median, std
def calc_stats(arr):
    mean = np.mean(arr)
    median = np.median(arr)
    std = np.std(arr, ddof=1)  # sample std dev
    return mean, median, std

attention_stats = calc_stats(attention)
relevance_stats = calc_stats(relevance)
confidence_stats = calc_stats(confidence)
satisfaction_stats = calc_stats(satisfaction)
perceived_learning_stats = calc_stats(perceived_learning)

attention_stats, relevance_stats, confidence_stats, satisfaction_stats, perceived_learning_stats

((np.float64(4.15), np.float64(4.125), np.float64(0.5027701042999452)),
 (np.float64(4.2), np.float64(4.125), np.float64(0.6324555320336759)),
 (np.float64(3.3), np.float64(3.25), np.float64(0.2838231060987733)),
 (np.float64(4.375), np.float64(4.375), np.float64(0.44487826050130463)),
 (np.float64(3.95), np.float64(3.75), np.float64(0.7527726527090811)))

In [10]:
#T-Tests

import numpy as np
from scipy import stats

# Data: question -> (before %, after %)
data = {
    "Q1": (0, 20),
    "Q2": (10, 10),
    "Q3": (0, 30),
    "Q4": (10, 70),
    "Q5": (10, 30),
    "Q6": (20, 60),
    "Q7": (60, 100),
    "Q8": (30, 30),
    "Q9": (0, 30),
    "Q10": (70, 90),
    "Q11": (0, 80),
    "Q12": (80, 90),
    "Q13": (70, 90),
}

# Define groups
groups = {
    "General Cybersecurity Tools": ["Q7", "Q8", "Q12"],
    "Network Protocols and Ports": ["Q2", "Q3", "Q10", "Q13"],
    "General Knowledge": ["Q5", "Q6", "Q9"],
    "Advanced Vulnerability Assessment and Penetration Testing Tools": ["Q1", "Q4", "Q11"],
}

# Convert % to decimals
def to_decimal(pct):
    return pct / 100

def one_tailed_p(t_stat, two_tailed_p):
    # If t_stat > 0, p_one_tailed = two_tailed_p / 2, else 1 - two_tailed_p / 2 (or just 1 because no gain)
    if t_stat > 0:
        return two_tailed_p / 2
    else:
        return 1 - two_tailed_p / 2

# Overall stats
before_all = np.array([to_decimal(data[q][0]) for q in data])
after_all = np.array([to_decimal(data[q][1]) for q in data])
gain_all = after_all - before_all
mean_before_all = np.mean(before_all)
mean_after_all = np.mean(after_all)
mean_gain_all = np.mean(gain_all)
t_stat_all, p_val_one_tailed_all = stats.ttest_rel(after_all, before_all)
p_val_one_tailed_all = one_tailed_p(t_stat_all, p_val_one_tailed_all)
sd_all = np.std(gain_all, ddof=1)

print(f"Overall: Mean before={mean_before_all*100:.2f}%, after={mean_after_all*100:.2f}%, gain={mean_gain_all*100:.2f}%")
print(f"t = {t_stat_all:.3f}, SD = {sd_all*100:.2f}%, p (1-tailed) = {p_val_one_tailed_all:.4f}")

# Per group stats
for group_name, questions in groups.items():
    before = np.array([to_decimal(data[q][0]) for q in questions])
    after = np.array([to_decimal(data[q][1]) for q in questions])
    gain = after - before
    mean_before = np.mean(before)
    mean_after = np.mean(after)
    mean_gain = np.mean(gain)
    sd = np.std(gain, ddof=1)
    t_stat, p_val_two_tailed = stats.ttest_rel(after, before)
    p_val_one_tailed = one_tailed_p(t_stat, p_val_two_tailed)

    print(f"\nGroup: {group_name}")
    print(f"Mean before = {mean_before*100:.2f}%, Mean after = {mean_after*100:.2f}%, Mean gain = {mean_gain*100:.2f}%")
    print(f"t = {t_stat:.3f}, SD = {sd*100:.2f}%, p (1-tailed) = {p_val_one_tailed:.4f}")

Overall: Mean before=27.69%, after=56.15%, gain=28.46%
t = 4.526, SD = 22.67%, p (1-tailed) = 0.0003

Group: General Cybersecurity Tools
Mean before = 56.67%, Mean after = 73.33%, Mean gain = 16.67%
t = 1.387, SD = 20.82%, p (1-tailed) = 0.1499

Group: Network Protocols and Ports
Mean before = 37.50%, Mean after = 55.00%, Mean gain = 17.50%
t = 2.782, SD = 12.58%, p (1-tailed) = 0.0345

Group: General Knowledge
Mean before = 10.00%, Mean after = 40.00%, Mean gain = 30.00%
t = 5.196, SD = 10.00%, p (1-tailed) = 0.0175

Group: Advanced Vulnerability Assessment and Penetration Testing Tools
Mean before = 3.33%, Mean after = 56.67%, Mean gain = 53.33%
t = 3.024, SD = 30.55%, p (1-tailed) = 0.0471
